# Lab 1 – Các Backend Deep Learning cho Phân Đoạn Làn Đường 
---
## Overview

Lab này giới thiệu **các backend deep learning cho bài toán phân đoạn làn đường** trong hệ thống AutoCar-Kit Lane Keeping.

Người học sẽ tìm hiểu cách tích hợp nhiều mô hình phân đoạn hiện đại như các **module perception có thể thay thế lẫn nhau**, cùng tạo ra đầu ra thống nhất là mặt nạ làn đường nhị phân.

Đầu ra của lab này đóng vai trò **nền tảng cho toàn bộ pipeline phía sau**, bao gồm lọc ROI, biến đổi BEV, ước lượng hình học làn đường và điều khiển đánh lái.

---

Trong pipeline tổng của đồ án, khối đầu tiên là:

> **Deep Learning Backend Segmentation** (YOLOv8 / PIDNet / TwinLiteNet / BiSeNetV2)  
> → sinh ra **binary lane mask `mask01`**

Lab 1 tập trung vào backend:

- Khởi tạo đúng các backend lane segmentation.
- Gọi `infer_mask01(frame_bgr)` để sinh `mask01`.
- Visualize kết quả phân đoạn.
- Lưu `Lab1_mask01_*.png` cho 10 frame (dùng lại ở Lab 2).

> Đây là bản **cho sinh viên**: các đoạn code quan trọng được ẩn bằng `...` + gợi ý chung.  
> Bạn cần tự đọc code gốc trong thư mục `AI/` để hoàn thiện.



# Gợi ý đọc code:


- Tìm hàm `build_lane()` trong `AI/main.py` để xem cách project khởi tạo backend.
- Xem các tham số cấu hình của từng model trong `AI/configs/config.py`.
- Trong Task 4–5–6, phương thức infer bạn cần dùng chính là phương thức
  mà project đang dùng để sinh `mask01` trước khi đưa sang ROI.


## Mục tiêu học tập

- Hiểu cấu trúc 4 backend lane segmentation của dự án.
- Khởi tạo được backend từ `model_name`.
- Sử dụng `infer_mask01` để tạo binary lane mask.
- So sánh trực quan kết quả của từng backend.
- Chuẩn bị file `Lab1_mask01_*.png` cho Lab 2.


# 0. Chuẩn bị môi trường


In [1]:
# 0.1. Import thư viện cơ bản
import sys
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
# 0.2. Thiết lập đường dẫn tới thư mục AI và import cấu hình + backend

ROOT = Path("../..").resolve()
AI_ROOT = ROOT / "AI"
sys.path.append(str(AI_ROOT))

print("Project ROOT:", ROOT)
print("AI ROOT     :", AI_ROOT)

from configs import config as C

from LaneDetection.backends.yolov8_backend import YoloV8Backend
from LaneDetection.backends.pidnet_backend import PIDNetBackend
from LaneDetection.backends.twinlite_backend import TwinLiteBackend
from LaneDetection.backends.bisenetv2_backend import BiseNetV2Backend

print("LANE_MODEL  (default):", C.LANE_MODEL)
print("LANE_WEIGHTS(default):", C.LANE_WEIGHTS)


# Nếu máy chỉ có CPU thì hãy chạy cell dưới đây để ép dùng CPU 

In [ ]:
import torch

print("Before override, C.DEVICE =", C.DEVICE)
print("torch.cuda.is_available():", torch.cuda.is_available())
C.DEVICE = "cpu"

print("After override,  C.DEVICE =", C.DEVICE)


# 1. Xây dựng bankend deep learning


###  Tổng quan

Ở pipeline thật của dự án, module backend đóng vai trò “bộ não perception”: nó nhận ảnh thô từ camera và trả về mask lane nhị phân.
File AI/main.py cho bạn thấy cách hệ thống thực sự tạo ra backend tương ứng với model đang chọn (YOLOv8 / PIDNet / TwinLite / BiSeNet).
Tất cả backend đều được xây dựng từ config và weight:

- Config chứa thông số kỹ thuật → input size, số class, threshold, kiến trúc PIDNet/TwinLite.

- Weight chứa learned parameters của model.

Khi viết build_backend(model_name) trong Lab này, bạn đang mô phỏng lại quy trình khởi tạo backend của hệ thống thực.

###  Để hoàn thành, bạn cần hiểu 3 điều:

**1) AI/main.py → Cách hệ thống thật tạo backend**

Tìm hàm build_lane() để thấy:

- Nó kiểm tra `LANE_MODEL`

- Map model → đúng class backend (YoloV8Backend, PIDNetBackend, v.v.)

- Truyền tham số từ config vào class

> ➡ Đây chính là logic bạn cần tái tạo trong Lab.

**2) AI/configs/config.py → Tìm tham số model**

Trong này bạn sẽ tìm thấy:

- `PIDNET_H`, `PIDNET_W`, `PIDNET_THR`, `PIDNET_ARCH`

- `TWIN_H`, `TWIN_W`, `TWIN_THR`, `TWIN_NUM_CLASSES`

- `IMGSZ`, `CONF` cho YOLOv8

- `BISENET_*` cho BiSeNet

>➡ Đây là “nguyên liệu” bạn phải truyền vào khi khởi tạo backend.

**3) AI/LaneDetection/Lane_weight/ → File weight tương ứng**

Xem thư mục này để biết:

- mỗi model dùng file weight nào

- đúng tên thư mục

- đúng phần mở rộng .pt hay .pth.

>➡ Bạn đang mô phỏng chính xác cách pipeline thật tìm file weight khi khởi chạy.



In [ ]:
# 1.1. Từ điển: tên model -> tên file weight tương ứng trong thư mục Lane_weight
LANE_WEIGHT_NAME = {
    "yolov8":  "Yolo_v8/best.pt",
    "pidnet":  "PIDNet/best.pt",
    "twinlite":"TwinLite/best.pth",
    "bisenet": "BiseNet/best.pth",
}

def get_lane_weight_path(model_name: str) -> Path:
    """
    Trả về đường dẫn đầy đủ tới file weight cho model_name.

    Gợi ý chung:
    - Thư mục gốc chứa weight nằm trong thư mục LaneDetection/Lane_weight của dự án.
    - Tên file cụ thể bạn có thể lấy từ dictionary LANE_WEIGHT_NAME ở trên.
    """
    name = model_name.lower()
    if name not in LANE_WEIGHT_NAME:
        raise ValueError(f"Unknown lane model: {model_name}")

    # TODO: ghép đường dẫn thư mục weight với tên file tương ứng.
    # Bạn cần tự quyết định dùng những thành phần nào từ biến C.ROOT.
    weights_dir = ...   # Ví dụ: đường dẫn tới thư mục chứa tất cả file weight
    weight_rel  = ...   # Ví dụ: tên file weight lấy từ dictionary LANE_WEIGHT_NAME
    weight_path = ...   # Ghép thư mục + tên file để được đường dẫn đầy đủ

    return weight_path


In [ ]:
def build_backend(model_name: str):
    """
    Khởi tạo backend lane segmentation tương ứng.

    Parameters
    ----------
    model_name : str
        'yolov8' | 'pidnet' | 'twinlite' | 'bisenet'
    """
    name = model_name.lower()
    weights = get_lane_weight_path(name)

    # TODO: hoàn thiện từng nhánh if/elif.
    # Gợi ý chung:
    #  - Các tham số cấu hình (kích thước input, ngưỡng, số lớp, ...) đã được định nghĩa trong file config của dự án.
    #  - Hãy mở configs/config.py để xem tên các tham số tương ứng với từng model.
    if name == "yolov8":
        backend = YoloV8Backend(
            weights=str(weights),
            device=C.DEVICE,
            imgsz=...,   # Gợi ý: kích thước ảnh đầu vào dùng chung cho YOLO
            conf=...,    # Gợi ý: ngưỡng confidence của YOLO
        )
    elif name == "pidnet":
        backend = PIDNetBackend(
            weights=str(weights),
            device=C.DEVICE,
            input_h=...,   # Gợi ý: chiều cao input của PIDNet
            input_w=...,   # Gợi ý: chiều rộng input của PIDNet
            thr=...,       # Gợi ý: ngưỡng phân đoạn của PIDNet
            arch=...,      # Gợi ý: lựa chọn kiến trúc của PIDNet
        )
    elif name == "twinlite":
        backend = TwinLiteBackend(
            weights=str(weights),
            device=C.DEVICE,
            input_h=...,          # Gợi ý: chiều cao input của TwinLite
            input_w=...,          # Gợi ý: chiều rộng input của TwinLite
            thr=...,              # Gợi ý: ngưỡng phân đoạn của TwinLite
            num_classes=...,      # Gợi ý: số lớp đầu ra của TwinLite
        )
    elif name == "bisenet":
        backend = BiseNetV2Backend(
            weights=str(weights),
            device=C.DEVICE,
            input_h=...,          # Gợi ý: chiều cao input của BiSeNetV2
            input_w=...,          # Gợi ý: chiều rộng input của BiSeNetV2
            num_classes=...,      # Gợi ý: số lớp đầu ra của BiSeNetV2
        )
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    return backend


In [ ]:
# 1.2. Sau khi điền TODO, chạy cell này để kiểm tra nhanh.
for name in ["yolov8", "pidnet", "twinlite", "bisenet"]:
    try:
        b = build_backend(name)
        print(f"{name:8s} ->", b.__class__.__name__)
    except Exception as e:
        print(f"{name:8s} -> ERROR:", e)


Sau khi đọc các file trên, bạn sẽ hiểu từ model_name → tìm weight → lấy config → tạo backend là một quy trình chuẩn, giống như “chọn module perception” trong xe tự hành thật.

# 2. Load các frame mẫu


###  Tổng quan

Trước khi đưa ảnh vào backend, bạn phải đảm bảo rằng ảnh đã được load đúng, vì mọi bước sau đều phụ thuộc vào chất lượng ảnh đầu vào.
Task này giúp bạn hiểu cách pipeline thật nhận frame từ camera và xử lý từng frame.


###  Để hoàn thành:

**1) Hiểu đúng cấu trúc thư mục frame**

Bạn phải xác định chính xác:

- Thư mục chứa frame nằm ở đâu

- Pattern file là gì (*.jpg, *.png, …)

- Tên file có dạng gì (Lab1_frames_01.png, frame_001.jpg…)

>➡ Vì glob() sẽ không tìm được frame nếu pattern sai → số frame = 0.

**2) Dùng OpenCV để đọc ảnh**

cv2.imread() đọc ảnh dưới dạng BGR, là format mà backend yêu cầu.
Bạn cần kiểm tra:

- ảnh có load được không

- shape ảnh có đúng không (H, W, 3)

- Giống pipeline thật, việc kiểm tra 1–2 frame đầu giúp đảm bảo camera input hợp lệ.



In [ ]:
FRAMES_DIR = ROOT / "Lab" / "Lab1" / "Lab1_frames"

frame_paths = sorted(FRAMES_DIR.glob("frame_*.jpg"))
print("Số frame tìm được:", len(frame_paths))
for p in frame_paths[:5]:
    print(" -", p)


In [ ]:
# Xem thử một frame
if len(frame_paths) == 0:
    raise RuntimeError("Không tìm thấy frame nào trong data/lab1_frames")

sample_path = frame_paths[0]
frame_bgr = cv2.imread(str(sample_path))
if frame_bgr is None:
    raise FileNotFoundError(f"Không đọc được ảnh: {sample_path}")

frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,4))
plt.imshow(frame_rgb)
plt.title(sample_path.name)
plt.axis("off")
plt.show()


Sau khi hoàn thành, bạn nắm được cách pipeline thật load frame từ camera → đây là bước nền tảng để backend xử lý chính xác.

# 3. Xây dựng hàm hiển thị frame + mask lane


###  Tổng quan

Trong project thật, lane mask được overlay lên ảnh để debug và visualization.
Overlay giúp bạn nhìn thấy:

- model phân đoạn lane như thế nào

- mask có liền mạch hay bị noise

- lane có khớp với đường thực tế không.

- Hàm visualize trong Task này mô phỏng chính xác logic overlay của pipeline thật trong main.py hoặc `lane_pipeline.py`.


###  Để viết hàm show_frame_and_mask:

**1) Chuyển ảnh sang RGB**

Matplotlib hiển thị ảnh theo RGB → bạn phải chuyển từ BGR sang RGB.

**2) Tạo lớp overlay**

- Copy frame gốc

- Tô màu vùng mask (frame[mask>0] = [255,0,0])

- Chọn màu lane tùy ý

**3) Dùng cv2.addWeighted**

Tạo frame trong suốt, giúp nhìn thấy cả lane + background.
Pipeline thật dùng đúng kỹ thuật này.


In [ ]:
def show_frame_and_mask(frame_bgr, mask01, title=""):
    """Hiển thị frame gốc và frame có overlay lane."""
    # TODO: chuyển từ BGR (OpenCV) sang RGB (matplotlib)
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # TODO: tạo một bản copy và tô màu vùng lane (mask01 == 1)
    overlay = frame_rgb.copy()
    overlay[mask01 > 0] = ...   # Gợi ý: dùng một màu nổi bật như đỏ thuần

    # TODO: trộn frame_rgb và overlay bằng addWeighted để tạo hiệu ứng trong suốt
    alpha = 0.5
    blended = cv2.addWeighted(frame_rgb, 1 - alpha, overlay, alpha, 0)

    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.imshow(frame_rgb)
    plt.title("Frame gốc")
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.imshow(blended)
    plt.title(title)
    plt.axis("off")

    plt.tight_layout()
    plt.show()


Bạn hiểu rõ cơ chế visualization của pipeline lane detection và biết cách overlay mask chuẩn như project thật.

# 4. Chạy thử một backend (TwinliteNet)


### Tổng quan

Mỗi backend có cách infer khác nhau: resize input, normalize, đổi thứ tự kênh màu.
Tuy nhiên backend class đã “gói" hết vào một hàm:

`backend.infer_mask01(frame_bgr)`


Pipeline thật cũng chỉ gọi đúng hàm này.
Task 4 giúp bạn thấy cách một backend hoạt động độc lập.


### Để hoàn thành bạn cần:

**1) Xác định hàm infer trong backend file**

Ví dụ:

- YOLOv8: infer_mask01()

- PIDNet: infer_mask01()

- TwinLite: infer_mask01()

- BiSeNet: infer_mask01()

Chúng đều trả về mask.

**2) Xem input yêu cầu**

Backend thường nhận BGR ảnh gốc (đã resize bên trong class).

**3) Xem output mask**

Output có thể là:

- 0–255

- hoặc 0–1

Nhưng bạn luôn ép về 0–1 trong Lab để thống nhất:

`(mask > 0).astype(np.uint8)`



In [ ]:
backend = build_backend("twinlite") #  đổi tên model nếu muốn dùng bankend khác (Yolov8, BiseNet, PidNet)

sample_path = frame_paths[0]
frame_bgr = cv2.imread(str(sample_path))

# TODO: gọi hàm infer của backend để lấy mask phân đoạn lane
mask = ...      # Gợi ý: dùng một phương thức của backend với input là frame_bgr

# TODO: chuyển mask thành mask01 dạng 0/1 (True/False -> 1/0)
mask01 = ...    

print("Frame shape:", frame_bgr.shape)
print("Mask  shape:", getattr(mask, "shape", None))

show_frame_and_mask(frame_bgr, mask01, title="twinlite – mask01") 


Bạn hiểu rằng backend là “hộp đen perception”: chỉ cần cho ảnh vào → lấy mask ra.
Đây là đúng logic mà pipeline thật dùng.

# 5. So sánh 4 backend trên cùng một frame


### Tổng quan

Nhiệm vụ này mô phỏng bài toán benchmark model trong thực tế:
- cùng 1 frame → chạy qua 4 model khác nhau → so sánh chất lượng và độ ổn định.

- Pipeline thật của nhóm nghiên cứu thường làm giống hệt.


### Để hoàn thành:

**1) Reuse code từ nhiệm vụ 3 & niệm vụ  4**

- Nhiệm vụ 3: dùng lại hàm overlay

- Nhiệm vụ 4: dùng build_backend(name) và infer_mask01(...)

**2) Loop qua danh sách model**
["yolov8", "pidnet", "twinlite", "bisenet"]

**3) Visualize từng backend**

Đặt tiêu đề rõ ràng: YOLOv8 / PIDNet / TwinLite / BiSeNet.



In [ ]:
model_names = ["yolov8", "pidnet", "twinlite", "bisenet"]

sample_path = frame_paths[0]
frame_bgr = cv2.imread(str(sample_path))

plt.figure(figsize=(12, 6))

for i, name in enumerate(model_names, start=1):
    # TODO: khởi tạo backend tương ứng
    backend = ...

    # TODO: infer mask và chuyển sang mask01 0/1
    mask = ...
    mask01 = ...

    # TODO: tạo overlay từ mask01 (có thể tái sử dụng logic ở Task 3)
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    overlay = frame_rgb.copy()
    overlay[mask01 > 0] = [255, 0, 0]
    blended = cv2.addWeighted(frame_rgb, 0.5, overlay, 0.5, 0)

    plt.subplot(2, 2, i)
    plt.imshow(blended)
    plt.title(name.upper())
    plt.axis("off")

plt.tight_layout()
plt.show()


Bạn hiểu được điểm mạnh – điểm yếu của từng model và thấy được vì sao pipeline của xe tự hành cần backend mạnh.

# 6. Tạo và lưu mặt nạ làn đường nhị phân để phục vụ phân tích


### Tổng quan

Trong toàn bộ pipeline lane-keeping của project, mask01 (mask nhị phân) là đầu vào bắt buộc cho các bước sau:

- ROI selection

- Morphology

- BEV transform

- Lane geometry estimation

- Controller (EMA + steering output)

phần này giúp bạn tạo ra bộ mask01 chuẩn để dùng lại y hệt pipeline thật.


### Để làm hoàn thành:

**1) Loop qua tất cả frame**

Infer mask → ép về 0–1 → lưu vào list.

**2) Stack thành numpy array**
`all_masks = np.stack(all_masks, axis=0)`


Shape sẽ là: `(10, H, W)`

3) Lưu bằng numpy

.py dùng:

`np.save("lab1_mask01.npy", all_masks)`

Hoặc PNG (phiên bản bạn dùng)

- mask01 → 0–1

- multiply 255 → PNG

- lưu từng file


In [ ]:

# MODEL_FOR_LAB2 = "twinlite"  # hoặc 'pidnet' / 'yolov8' / 'bisenet'
MODEL_FOR_LAB2 = "twinlite" # chọn backend (Yolov8, BiseNet, PidNet)

backend = build_backend(MODEL_FOR_LAB2)

# TODO: tạo thư mục để lưu output PNG
OUT_DIR = ROOT / "Lab" / "Lab1" / "Lab1_masks"
OUT_DIR.mkdir(parents=True, exist_ok=True)

for i, p in enumerate(frame_paths, start=1):
    frame_bgr = cv2.imread(str(p))
    if frame_bgr is None:
        raise FileNotFoundError(f"Không đọc được ảnh: {p}")

    # TODO: infer mask từ backend
    mask = ...

    # TODO: chuyển mask sang nhị phân 0–1 (uint8)
    mask01 = ...

    # TODO: nhân 255 để lưu ảnh PNG (0–1 → 0–255)
    mask_png = ...

    out_path = OUT_DIR / f"Lab1_mask01_{i:02d}.png"

    cv2.imwrite(str(out_path), mask_png)

print("➡ Đã hoàn thành sinh mask PNG cho 10 frame!", out_path)


Bạn đã hoàn thành phần perception của pipeline lane detection.
Từ đây, Lab 2 & Lab 3 sẽ dùng mask này để xây dựng ROI → BEV → Control như trong dự án thực.

# 7. Tổng kết

Khi hoàn thành tất cả TODO, bạn đã:

- Khởi tạo được backend tương ứng với từng model bằng cách đọc và hiểu file config của dự án.
- Dùng hàm infer của backend để tạo binary lane mask.
- So sánh trực quan 4 backend trên cùng một frame.
- Sinh file `Lab1_mask01_*.png` để phục vụ Lab 2.

👉 Đừng quên giữ lại những file `.png` trong thư mục `/Lab/Lab1_mask/Lab1_mask01_01.png ... Lab1_mask01_10 `.
